<a href="https://colab.research.google.com/github/jugalpanchal/bd-chef/blob/main/spark_rdd_and_df.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Follow the steps to install the dependencies:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null # install java
!wget -q https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz # spark package download
!tar xf spark-3.1.2-bin-hadoop3.2.tgz # unzip spark package
!pip install -q findspark # install spark

# Set the location of Java and Spark:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import Row

# create or get spark session
spark = SparkSession.builder \
        .master("local[*]") \
        .appName("Spark_App1") \
        .getOrCreate()

# Spark 2.x does not give direct sparkcontext so we need to get it from SparkSession
sc = spark.sparkContext

### RDD

In [3]:
# Read a file and create a RDD - RDD Started in Spark 1.x
collection_rdd = sc.textFile("sample_data/anscombe.json")
#collection_rdd = sc.textFile("sample_data/*.json") # multiple files

collection_rdd

sample_data/anscombe.json MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [4]:
#collection_rdd.count() #49
#collection_rdd.take(5) # top 5 entities

In [5]:
collection_rdd.collect() # this is like a collection

['[',
 '  {"Series":"I", "X":10.0, "Y":8.04},',
 '  {"Series":"I", "X":8.0, "Y":6.95},',
 '  {"Series":"I", "X":13.0, "Y":7.58},',
 '  {"Series":"I", "X":9.0, "Y":8.81},',
 '  {"Series":"I", "X":11.0, "Y":8.33},',
 '  {"Series":"I", "X":14.0, "Y":9.96},',
 '  {"Series":"I", "X":6.0, "Y":7.24},',
 '  {"Series":"I", "X":4.0, "Y":4.26},',
 '  {"Series":"I", "X":12.0, "Y":10.84},',
 '  {"Series":"I", "X":7.0, "Y":4.81},',
 '  {"Series":"I", "X":5.0, "Y":5.68},',
 '',
 '  {"Series":"II", "X":10.0, "Y":9.14},',
 '  {"Series":"II", "X":8.0, "Y":8.14},',
 '  {"Series":"II", "X":13.0, "Y":8.74},',
 '  {"Series":"II", "X":9.0, "Y":8.77},',
 '  {"Series":"II", "X":11.0, "Y":9.26},',
 '  {"Series":"II", "X":14.0, "Y":8.10},',
 '  {"Series":"II", "X":6.0, "Y":6.13},',
 '  {"Series":"II", "X":4.0, "Y":3.10},',
 '  {"Series":"II", "X":12.0, "Y":9.13},',
 '  {"Series":"II", "X":7.0, "Y":7.26},',
 '  {"Series":"II", "X":5.0, "Y":4.74},',
 '',
 '  {"Series":"III", "X":10.0, "Y":7.46},',
 '  {"Series":"I

### DataFrame

In [6]:
# Read a file and create a DataFrame - DataFrame Started in Spark 2.x but on top of the RDD.
json_df = spark.read.json('sample_data/anscombe.json') # first row is always a columns row.
json_df

DataFrame[Series: string, X: double, Y: double, _corrupt_record: string]

In [7]:
json_df.printSchema()

root
 |-- Series: string (nullable = true)
 |-- X: double (nullable = true)
 |-- Y: double (nullable = true)
 |-- _corrupt_record: string (nullable = true)



In [8]:
json_df.collect()

[Row(Series=None, X=None, Y=None, _corrupt_record='['),
 Row(Series='I', X=10.0, Y=8.04, _corrupt_record=None),
 Row(Series='I', X=8.0, Y=6.95, _corrupt_record=None),
 Row(Series='I', X=13.0, Y=7.58, _corrupt_record=None),
 Row(Series='I', X=9.0, Y=8.81, _corrupt_record=None),
 Row(Series='I', X=11.0, Y=8.33, _corrupt_record=None),
 Row(Series='I', X=14.0, Y=9.96, _corrupt_record=None),
 Row(Series='I', X=6.0, Y=7.24, _corrupt_record=None),
 Row(Series='I', X=4.0, Y=4.26, _corrupt_record=None),
 Row(Series='I', X=12.0, Y=10.84, _corrupt_record=None),
 Row(Series='I', X=7.0, Y=4.81, _corrupt_record=None),
 Row(Series='I', X=5.0, Y=5.68, _corrupt_record=None),
 Row(Series='II', X=10.0, Y=9.14, _corrupt_record=None),
 Row(Series='II', X=8.0, Y=8.14, _corrupt_record=None),
 Row(Series='II', X=13.0, Y=8.74, _corrupt_record=None),
 Row(Series='II', X=9.0, Y=8.77, _corrupt_record=None),
 Row(Series='II', X=11.0, Y=9.26, _corrupt_record=None),
 Row(Series='II', X=14.0, Y=8.1, _corrupt_record=N

### RDD to DataFrame and vice versa

In [9]:
generic_data = sc.parallelize([1, 'Alice', 50, 'Bob', 'Canton'])
#generic_data.toDF() # It does not work because it contains object with differnt types.

In [10]:
people_list = [('Alice', 25), ('Bob', 20), ('Charly', 30), ('Don', 15)]
people_rdd_pair = sc.parallelize(people_list) # create a RDD but with Tuple/Pair objects
#people_rdd_pair # ParallelCollectionRDD[2] at readRDDFromFile

people_rdd_pair.collect()

[('Alice', 25), ('Bob', 20), ('Charly', 30), ('Don', 15)]

In [11]:
people_df = people_rdd_pair.toDF() # Column names have been automatically generated and assigned. It infers data types too.
#people_df # DataFrame[_1: string, _2: bigint]
people_df.collect() # shows data with its class

[Row(_1='Alice', _2=25),
 Row(_1='Bob', _2=20),
 Row(_1='Charly', _2=30),
 Row(_1='Don', _2=15)]

In [12]:
people_df.show() # shows in tabular format

+------+---+
|    _1| _2|
+------+---+
| Alice| 25|
|   Bob| 20|
|Charly| 30|
|   Don| 15|
+------+---+



In [13]:
from pyspark import Row
# There are two way to get custom column names.
# 1. Create the row class object while creating a RDD
data_rdd_row = sc.parallelize([Row(name='Alice', age=50), 
                               Row(name='Bob', age=20)])
data_rdd_row.collect() # [Row(name='Alice', age=50), Row(name='Bob', age=20)]

[Row(name='Alice', age=50), Row(name='Bob', age=20)]

In [14]:
# 2. Transform existing RDD to new RDD
people_rdd_row = people_rdd_pair.map(lambda x: Row(name=x[0], age=int(x[1]))) # create a RDD with Row objects
people_rdd_row # PythonRDD[63] at RDD
people_rdd_row.collect()

[Row(name='Alice', age=25),
 Row(name='Bob', age=20),
 Row(name='Charly', age=30),
 Row(name='Don', age=15)]

In [15]:
data_rdd_row.toDF().show()
people_rdd_row.toDF().show()

+-----+---+
| name|age|
+-----+---+
|Alice| 50|
|  Bob| 20|
+-----+---+

+------+---+
|  name|age|
+------+---+
| Alice| 25|
|   Bob| 20|
|Charly| 30|
|   Don| 15|
+------+---+



In [16]:
people_df.rdd # Every DataFrame contains a RDD and we can get the dataframe's RDD easily.

MapPartitionsRDD[39] at javaToPython at NativeMethodAccessorImpl.java:0

In [17]:
people_df = data_rdd_row.toDF()

### Operation on DataFrame

In [18]:
# map operation on RDD
people_rdd = people_df.rdd.map(lambda x: (x.name, x.age + 5))
#people_rdd # PythonRDD[78] at RDD
people_rdd.collect()

[('Alice', 55), ('Bob', 25)]

In [19]:
# Map: the map operation performs a transformation on every element in the RDD. 
# Similar kind of method is not available for the DataFrame, 
# so we compute and assign computed value in new column within the DataFrame. 

people_df_new_age = people_df.select('name', 'age').withColumn('new_age', people_df.age + 10) # we can type people_df.age to mention column name.
people_df_new_age.show()

people_df_new_age = people_df_new_age.withColumnRenamed('new_age', 'after_ten_years')
people_df_new_age = people_df_new_age.drop('age')
people_df_new_age.show()

+-----+---+-------+
| name|age|new_age|
+-----+---+-------+
|Alice| 50|     60|
|  Bob| 20|     30|
+-----+---+-------+

+-----+---------------+
| name|after_ten_years|
+-----+---------------+
|Alice|             60|
|  Bob|             30|
+-----+---------------+



In [20]:
people_df.first()

Row(name='Alice', age=50)

In [21]:
people_df.take(2)

[Row(name='Alice', age=50), Row(name='Bob', age=20)]

In [22]:
people_df.collect()[1][0] # It is a tabular format so can access specific cell.
# It always creates a clone then return so if the cell has any class type and we modify then it won't change in the dataframe.

'Bob'

###Inferred Schemas

In [23]:
lines_rdd = sc.textFile("datasets/students.txt")
lines_rdd.collect()

['Emily,44,55,78', 'Andy,47,34,89', 'Rick,55,78,55', 'Aaron,66,34,98']

In [24]:
parts_rdd = lines_rdd.map(lambda l: l.split(","))
parts_rdd.collect()

[['Emily', '44', '55', '78'],
 ['Andy', '47', '34', '89'],
 ['Rick', '55', '78', '55'],
 ['Aaron', '66', '34', '98']]

In [25]:
students_rdd = parts_rdd.map(lambda p: Row(name=p[0], math=int(p[1]), english=int(p[2]), science=int(p[3])))
students_rdd.collect()

[Row(name='Emily', math=44, english=55, science=78),
 Row(name='Andy', math=47, english=34, science=89),
 Row(name='Rick', math=55, english=78, science=55),
 Row(name='Aaron', math=66, english=34, science=98)]

In [26]:
students_df = spark.createDataFrame(students_rdd)
students_df.createOrReplaceTempView("students_tbl")
students_df.columns

['name', 'math', 'english', 'science']

In [27]:
students_df.schema # Spark generates schema automatically.

StructType(List(StructField(name,StringType,true),StructField(math,LongType,true),StructField(english,LongType,true),StructField(science,LongType,true)))

In [28]:
spark.sql("SELECT * FROM students_tbl").show()

+-----+----+-------+-------+
| name|math|english|science|
+-----+----+-------+-------+
|Emily|  44|     55|     78|
| Andy|  47|     34|     89|
| Rick|  55|     78|     55|
|Aaron|  66|     34|     98|
+-----+----+-------+-------+



 ### Explicit Schemas

In [29]:
parts_rdd.collect()

[['Emily', '44', '55', '78'],
 ['Andy', '47', '34', '89'],
 ['Rick', '55', '78', '55'],
 ['Aaron', '66', '34', '98']]

In [30]:
from pyspark.sql.types import StructType, StructField, StringType, LongType

fields = [StructField('name', StringType(), True),
          StructField('math', LongType(), True),
          StructField('english', LongType(), True),
          StructField('science', LongType(), True),
]
student_schema = StructType(fields)

students_explicit_df = spark.createDataFrame(parts_rdd, student_schema)
students_explicit_df.columns

['name', 'math', 'english', 'science']

In [31]:
students_explicit_df.schema

StructType(List(StructField(name,StringType,true),StructField(math,LongType,true),StructField(english,LongType,true),StructField(science,LongType,true)))